# AGE GENDER 

In [12]:
import os
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

In [13]:
folder_path = r'D:\copy of htdocs\practice\Python\200days\Day195 Deep Learning Day 25\utkface_aligned_cropped\UTKFace'

In [14]:
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [15]:
len(age)

128

In [16]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [17]:
df.shape

(128, 3)

In [18]:
df.head()

,age,gender,img
0,1,0,1_0_0_20161219140623097.jpg.chip.jpg
1,1,0,1_0_0_20161219140627985.jpg.chip.jpg
2,1,0,1_0_0_20161219140642920.jpg.chip.jpg
3,1,0,1_0_0_20161219154018476.jpg.chip.jpg
4,1,0,1_0_0_20161219154556757.jpg.chip.jpg


In [19]:
train_df = df.sample(frac=1,random_state=0).iloc[:100]
test_df = df.sample(frac=1,random_state=0).iloc[100:]

In [20]:
train_df.shape

(100, 3)

In [21]:
test_df.shape

(28, 3)

In [22]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [23]:
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                    class_mode='multi_output')

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                  class_mode='multi_output')

Found 100 validated image filenames.
Found 28 validated image filenames.


In [24]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [25]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

94765736/94765736 [==============================] - 72s 1us/step


In [26]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [27]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [30]:
history=model.compile(optimizer='adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})

In [31]:
model.fit(train_generator, batch_size=32, epochs=10, validation_data=test_generator)

Epoch 1/10
4/4 [==============================] - 12s 2s/step - loss: 9.6887 - age_loss: 9.6887 - gender_loss: 0.0000e+00 - age_mae: 9.6887 - gender_accuracy: 1.0000 - val_loss: 3.3496 - val_age_loss: 3.3496 - val_gender_loss: 0.0000e+00 - val_age_mae: 3.3496 - val_gender_accuracy: 1.0000
Epoch 2/10
4/4 [==============================] - 8s 2s/step - loss: 3.0442 - age_loss: 3.0442 - gender_loss: 0.0000e+00 - age_mae: 3.0442 - gender_accuracy: 1.0000 - val_loss: 2.6918 - val_age_loss: 2.6918 - val_gender_loss: 0.0000e+00 - val_age_mae: 2.6918 - val_gender_accuracy: 1.0000
Epoch 3/10
4/4 [==============================] - 8s 2s/step - loss: 1.5150 - age_loss: 1.5150 - gender_loss: 0.0000e+00 - age_mae: 1.5150 - gender_accuracy: 1.0000 - val_loss: 2.5385 - val_age_loss: 2.5385 - val_gender_loss: 0.0000e+00 - val_age_mae: 2.5385 - val_gender_accuracy: 1.0000
Epoch 4/10
4/4 [==============================] - 9s 2s/step - loss: 1.6827 - age_loss: 1.6827 - gender_loss: 0.0000e+00 - age_mae: 